# Introduction to Bioinformatics

## 03 - Multiple sequences alignment

In the second practice session, we learned how to create functions to wrap repetitive tasks and reuse this code. We created a function that read a fasta file containing multiple sequences into a Python dictionary and another function that calculated the Percentage of Identity between two aligned sequences. Then we moved to use a specialized library, called BioPython, to read fasta files and access the sequence information. Finally, we learn how to employ NumPy to create an array of numbers to store a pair-wise comparison matrix between all the elements in a collection of sequences.

In this practical session, we will explore how we can call external programs from Python. We do this to generate multiple sequence alignments that can be read directly using BioPython. We will use the MAFFT program, a multiple sequence alignment program for Unix-like operating systems.  It offers a range of multiple alignment methods. Especially noteworthy is the INS-i, which is accurate for alignments of up to 200 sequences.

This notebook assumes that you have the program installed. If you have not installed it by any chance, please refer to the preparation instructions that appear in the README.md file in the practical session 03 GitHub's page.

### Introduction

When comparing biological sequences, there is a constant need to create an evolutionary relationship among the sequences under scrutiny. This comparison should reflect as close as possible the history of the genes involved, i.e., the mutations, insertion, and deletions that had happened as the genes diverged from their common point in biological history. To approximate this task, it is useful to observe conservation patterns among the sequences available to us in the present day. The objective of multiple sequence alignment is to create the maximum possible matching between the sequences to observe these conserved motifs.

The problem of sequence alignment is of higher complexity than that of the pair-wise alignment, and specialized algorithms are necessary to approach it efficiently and accurately.

Today we will employ the MAFFT program, which we usually would call from the command line (i.e., from the terminal). We will integrate the program into a python function able to call this program (e.g., from the notebook) and read the output conveniently using BioPython.

### Gathering the sequences to be compared

For this session, we will use a collection of sequences for the 2-hydroxymuconate tautomerase protein, one of the smallest enzymes known (close to 62 amino acid residues). First, we will [BLAST](https://www.uniprot.org/blast/) a representative sequence of this protein family, with code P70994, in the UniprotKB/Swissprot database. Then we download the fasta file that contains 250 entries (this file is already in the input folder of this practical session). For our purpose, the file now has too many sequences, and therefore we need to narrow it down to some significant sequences so our comparison would be more tractable and confident.

### Generating a set of representative sequences.

To generate groups of similar sequences, we are going to employ the CD-HIT program. This program can create very rapidly groups of related sequences based on a sequence identity threshold. We will use a very stringent sequence identity threshold to generate the groups (this will ensure we have the smallest possible set of sequences to compare).
To execute CD-HIT you need to go to the terminal and run the following command in the practical session directory:

```cd-hit -i input/2HDXMT_250.fasta -o output/2HDXMT_clusters.fasta -c 0.65```

This command can be broken down as follows:
	cd-hit: represents the command name
	-i: is the option to give the path to the input fasta file.
	-o: is the option to tell the path to the output fasta file.
	-c: is the option that defines the sequence identity threshold.

We will see two output files in the "output" folder: 2HDXMT_clusters.fasta and 2HDXMT_clusters.fasta.clstr. The former is the fasta file containing our non-redundant set of sequences no more similar between them than our selected sequence identity threshold of 0.65. The latter is a file with the summary of the assignment of the original 250 sequences in the just created 20 clusters. We will employ the 2HDXMT_clusters.fasta file for building our multiple sequence alignment.

### Adding left-out sequences

Despite creating a group of significant sequences, there is a good chance that our sequence of interest has been left out. This exclusion happens because the program selects the cluster centroid sequence to be written to the fasta file, and there is a small chance that our sequence is the centroid sequence of its cluster.

We did the BLAST search with code P70994; we can look at the 2HDXMT_clusters.fasta.clstr file to see in which sequence cluster our target sequence was grouped:

In [ ]:
with open('output/2HDXMT_clusters.fasta.clstr') as output:
    for line in output:
        print(line)

We note that our sequence is in cluster 0. CD-HIT puts an * after the centroid sequence. 

Can we find which are the centroid sequences for each cluster?

In [ ]:
with open('output/2HDXMT_clusters.fasta.clstr') as output:
    for line in output:
        if '*' in line or '>Cluster' in line:
            print(line)

The centroid sequence is the sequence with UniProt ID: B7GMK5. For us, it would be more convenient to replace this sequence with our target sequence, P70994. In this way, we could make any analysis containing the information of interest included. Let's do this with BioPython. 

Let's read all the sequences in the CD-HIT output fasta file and our target sequence, which is already in the input folder as P70994.fasta.

In [ ]:
from Bio import SeqIO 

In [ ]:
cdhit_sequences = SeqIO.parse('output/2HDXMT_clusters.fasta', 'fasta')
target_sequence = SeqIO.parse('input/P70994.fasta', 'fasta')

The SeqIO module also has a method for writing fasta files. It needs three arguments:

   - A list with the seqRecord objects
   - A file handler open in writing mode
   - The format in which the program should save the file
  
We create a list and save all but the B7GMK5 sequence, which we will replace with our target sequence.

In [ ]:
# Create a list to store the seqRecord objects to be written
selected = []

# Iterate the fasta iterator for our sequence
for seq_record in target_sequence:
    selected.append(seq_record)
    
# Iterate the fasta iterator for the CD-HIT clusters sequences
for seq_record in cdhit_sequences:
    
    # We append all but the sequence with ID B7GMK5
    if 'B7GMK5' not in seq_record.id:
        selected.append(seq_record)
        
# We call a handler to write the sequences into a file
with open('output/my_clusters.fasta', 'w') as output_file:
    
    # Write the sequences with BioPython SeqIO.write() method.
    SeqIO.write(selected, output_file, 'fasta')

Now that we have sequences we want to compare in a fasta file let us proceed with the multiple sequence alignment. 

### Calling a multiple sequence alignment (MSA) from the terminal with MAFFT

First, we practice with the program from the command line. To execute MAFFT, we need to go to the terminal and run:

 ```mafft --auto output/my_clusters.fasta```

The code is broken down as follows:
	mafft: represents the command name 
	--auto: is an option to select the alignment algorithm automatically, depending on our input fasta file.

The path to the input fasta file is given at the end of the command. 

We notice that the program does not write its output to a file, but it prints it directly on the terminal. To store the program's output, being written in the terminal screen, into a file, we employ the ">" character. This character is put at the end of a command to redirect the command's printed output into a specific file. We now change our code to store the multiple sequence alignment (MSA) result into a file:

```mafft --auto output/my_clusters.fasta > output/2HDXMT_msa.fasta```

We can now read the newly created file "output/2HDXMT_msa.fasta" containing our MSA. For this, we employ BioPython's module AlignIO.

### Loading an MSA into BioPython

First, we import BioPython's AlignIO module:

In [ ]:
from Bio import AlignIO

The AlignIO module allows us to get our fasta file directly into a BioPython's object class called MultipleSeqAlignment. Let's load our fasta file into BioPython and call print and help on the MultipleSeqAlignment object.

In [ ]:
alignment = AlignIO.read("output/2HDXMT_msa.fasta", "fasta")
print(type(alignment))
print(alignment)

In [ ]:
help(alignment)

Before we move into analyzing our BioPython's MSA object, we will wrap everything we did in just one python function. 

### Calling terminal command's from the Python interpreter

A particular library, called os, can be employed to call command executions from a Python interpreter (e.g., from the jupyter notebook, which is an interactive Python interpreter):

In [ ]:
import os

The os library has a system() method that takes a string as its mandatory positional argument. This string represents a command as if you were to execute it directly into the bash terminal. Therefore we can call the os.system('command') to execute something on the terminal. 

Let's make a test to run the 'mafft' command here:

In [ ]:
os.system('mafft --auto output/my_clusters.fasta > temporary_file.fasta')

Notice, that our output file has a different name "output/2HDXMT_msa_test.fasta". If you correctly executed the program, we should find this file in the "output" folder. Let's try to load it to see if this was successful:

In [ ]:
alignment = AlignIO.read("temporary_file.fasta", "fasta")
print(alignment)

Since we have already loaded the MSA result into the alignment variable, we can use the "os" library to remove the file. This library has a particular method for it.

In [ ]:
os.remove('temporary_file.fasta')

If we try to load the file again, Python will throw a "No such file or directory" error:

In [ ]:
AlignIO.read("temporary_file.fasta", "fasta")

Now we are ready to create a function that can automate the MSA execution and its loading into BioPython.

### Create a Python multiple sequence alignment function using MAFFT

Let's declare such function:

In [ ]:
def multipleSequenceAlignment(input_fasta):
    """
    Function that performs a multiple sequence alignment using 
    the MAFFT command-line program. The execution is called with 
    the "os" library directly into the terminal. The process creates
    a temporary file called 'temporary_file.fasta' that is deleted 
    after loading the MSA information with BioPython.
    
    Parameters
    ==========
    input_fasta : str
        Path to the input fasta file containing the sequences to align. 
        
    Returns
    =======
    msa : Bio.Align.MultipleSeqAlignment
        A BioPython's MSA object.
    """
    
    # Create the string representing the alignment command
    command = 'mafft --auto '+input_fasta+' > temporary_file.fasta'
    print(command)
    
    # Execute the alignment command
    os.system(command)
    
    # Load the output fasta file into BioPython's MSA object
    msa = AlignIO.read('temporary_file.fasta', 'fasta')
    
    # Remove the temporary output file
    if os.path.exists('temporary_file.fasta'):
        os.remove('temporary_file.fasta')
        
    # Return the MSA object
    
    return msa

In [ ]:
msa = multipleSequenceAlignment('output/my_clusters.fasta')
print(msa)

Nice! You can now write you our functions to call any program from the command line directly as a Python function.

### Analyzing an MSA

Now that we can quickly call a multiple sequence alignment let's analyze the one we have created for the 2-hydroxymuconate tautomerase protein.

Let's count the number of positions that are utterly conserved in the MSA:

In [ ]:
# Get the length of the alignment
alignment_lenght = msa.get_alignment_length()

# Create a list to store the conserved positions indexes
conserved_indexes = []
# Create a list to store the conserved positions letters
conserved_letters = []

# Get the number of sequences
n_sequences = len(msa)

# Iterate all the alignment index positions
for i in range(alignment_lenght):
    
    # Define the list to store all the letters in the MSA for position i
    letters = []
    
    # Iterate all the sequences in the MSA
    for seq_record in msa:
        
        # Define the current character 
        character = seq_record.seq[i]
        
        # Store only letter characters
        if character != '-':
            letters.append(character)
            
    # Define a set of unique letters
    letters_set = set(letters)
    # Store positions with only one letter in the set
    if len(letters_set) == 1:
        
        # Store positions that are present in all the sequences
        if len(letters) == n_sequences:
            
            #Append index to the list of conserved indexes
            conserved_indexes.append(i)
            #Append character to the list of conserved letters
            conserved_letters.append(letters[0])
            
print('The conserved position indexes are:')
print(conserved_indexes)
print(conserved_letters)

The first alignment position contains a methionine residue, which is generally conserved because it is the start codon in mRNA to protein translation. We also observe a proline and a couple of glycine residues, which control the protein's dynamic behavior. The remaining are a charged "E" and "Q" and "T," which can generate hydrogen bond contacts. 

We can search the literature for information about these specific positions; however, we need to map them back in our target sequence to define them clearly. This mapping has to be done by iterating the aligned sequence with UniProt ID P70994:

In [ ]:
# count variable to count the sequence positions
count = 0 

# Iterate over all the sequences in the MSA
for seq_record in msa:
    
    # Only work with the sequence of interest
    if 'P70994' in seq_record.id:
        
        # Iterate all the positions in the target sequence
        for i in range(len(seq_record)):
            
            # Count a sequence position for each character different than '-'
            if seq_record.seq[i] != '-':
                count += 1
                
            # Only print the conserved positions
            if i in conserved_indexes:
                
                # Print the target sequence's position amino acid identity and residue index
                print(seq_record.seq[i]+str(count))

### Are any of these positions described as relevant for the protein's function? 

We can search the UniProt database to find if any of these positions have been described for a relevant function. We see that two positions are mentioned for the P70994 sequence:

 - P2A mutation -> Absence of tautomerase activity using 2-hydroxymuconate. For 2-hydroxy-2,4-pentadienoate and phenylenolpyruvate, a strong decrease of catalytic efficiency and a 2-fold decrease of affinity is observed.
 
 - R12A mutation -> Absence of tautomerase activity using 2-hydroxymuconate. For 2-hydroxy-2,4-pentadienoate and phenylenolpyruvate, a strong decrease of catalytic efficiency and a 2-fold decrease of affinity is observed.
 
The position P2 is essential for catalysis and is absolutely conserved. Position R12, which is also crucial for catalysis, does not seem to be exclusively conserved. Let's have a look at the alignment for that particular position. If we repeat the above's code, we can see the alignment position that corresponds to the R12 residue:  

In [ ]:
# count variable to count the sequence positions
count = 0 

# Iterate over all the sequences in the MSA
for seq_record in msa:
    
    # Only work with the sequence of interest
    if 'P70994' in seq_record.id:
        
        # Iterate all the positions in the target sequence
        for i in range(len(seq_record)):
            
            # Count a sequence position for each character different than '-'
            if seq_record.seq[i] != '-':
                count += 1
                
            if count == 12:
                
                # Print the target sequence's position amino acid identity and residue index
                print(i, seq_record.seq[i]+str(count))

Now we can print all the letters in that position:

In [ ]:
# Define the list to store all the letters in the MSA for position 11
letters = []
    
# Iterate all the sequences in the MSA
for seq_record in msa:
        
    # Define the current character 
    character = seq_record.seq[11]
        
    # Store only letter characters
    if character != '-':
        letters.append(character)
    
# Define a set of unique letters
letters_set = set(letters)

print("List of all the letters in the MSA's position 11:")
print(letters)
print()
print("Set of unique letters:")
print(letters_set)
print()

# We calculate the frequency that each letter appear in the MSA
for l in letters_set:
    frequency = letters.count(l)/len(letters)
    print('Frequency for letter '+l+' in position 11:')
    print(frequency)
    print()

We see that the enzyme can also tolerate a lysine residue in that position; however, it does it with a lower preference. Therefore, not only exclusively conserved residues can be biochemically meaningful, but positions with similar amino acids can also have functional roles.

### Generating a comparison matrix with an MSA

Another useful application of a multiple sequence alignment program is the distance Matrix. In the previous session, we saw how to calculate an identity matrix by comparing multiple pairwise alignments. Now we can do it directly from our MSA object. Let's first import NumPy:

In [ ]:
import numpy as np

We can now recall our PID function from our previous practice session here. We need to make a small adjustment to the code to directly input the two already-aligned sequences from the MSA object.

In [ ]:
def calculatePID(sequence_i, sequence_j):
    
    count = 0 # Variable containing the count of matching characters
    # iterate
    for k in range(len(sequence_i)):
        p1 = sequence_i[k] # Position k of first alignment 
        p2 = sequence_j[k] # Position k of second alignment
    
        # Only count when we don't have '-' at any position.
        if p1 != '-' and p2 != '-':
            # Only when the sequences match
            if p1 == p2:
                count += 1
            
    # Get length of each sequence
    li = len([p for p in sequence_i if p != '-'])
    lj = len([p for p in sequence_j if p != '-'])
    
    # the PID uses the length of the shortest sequence to convert to percentage
    pid = count/min(li, lj)
    return pid

With this formula then we proceed to iterate our sequences with a double for loop. This double iteration ensures that all the sequences are compared to each other:

In [ ]:
def distanceMatrixFromMSA(msa):
    
    N = len(msa)
    M = np.zeros((N, N))
    
    for i in range(N):
        for j in range(N):
            
            seqi = msa[i].seq
            seqj = msa[j].seq
            
            if i == j: # the same element always has a PID=1.0
                M[i][j] = 1.0
            if j > i : # We compare only half matrix
                M[i][j] = calculatePID(seqi, seqj)
                M[j][i] = M[i][j] # This to fill the lower half matrix
    return M

Notice that we only need to calculate the PID for a portion of the matrix as in the previous practice session. This computational saving is because the diagonal always is filled with ones, and also because the comparison matrix is symmetric; comparing i to j and comparing j to i is the same.

We call the function, and then we plot the matrix using the matplotlib library. 

In [ ]:
M = distanceMatrixFromMSA(msa)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.matshow(M)
plt.title('PID matrix')
plt.xlabel('Sequence index i')
plt.ylabel('Sequence index j')

### Wrapping up

In this third practice session, we learned:

- How to generate a non-redundant and significant set of sequences with CD-HIT
- How to write sequences into a fasta file with BioPython
- How to use the MAFFT program for multiple sequence alignments (MSAs).
- How to execute terminal commands using the Python interpreter.
- How to read MSA files with BioPython
- How to operate an MSA to search for patterns.